# BaCP Ablation B3: Supervised vs. Self-Supervised Contrastive Loss Contribution

In [1]:
from bacp import BaCPTrainer, BaCPTrainingArguments
from trainer import Trainer, TrainingArguments
from utils import print_statistics, get_device, get_num_workers, print_dynamic_lambdas_statistics, load_object, save_object
from constants import *

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = 'resnet50'
MODEL_TASK = 'cifar10'
FINETUNED_WEIGHTS = './research/resnet50/cifar10/resnet50_cifar10_baseline.pt'
TRAIN = False
DEVICE = get_device()
NUM_WORKERS = get_num_workers()
print("Using device:", DEVICE)
print("Using", NUM_WORKERS, "workers")

Using device: cuda
Using 288 workers


## Baseline Check

In [3]:
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    scheduler_type='linear_with_warmup',
    epochs=100,
    learning_type="baseline",
    patience=50,
    db=False,
)
trainer = Trainer(training_args=training_args)
metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Linear scheduler initialized with warmup steps: 830 and total steps: 8300
[TRAINER] Pruning not initialized
[TRAINER] Saving model to: ./research/resnet50/cifar10/resnet50_cifar10_baseline.pt
[TRAINER] Loading weights: ./research/resnet50/cifar10/resnet50_cifar10_baseline.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.0



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     93.02%

Model Information:
------------------------------
  Total Parameters:     23,520,842
  Trainable Parameters: 23,520,842

Training Configuration:
------------------------------
  Model:                resnet50
  Task:                 cifar10
  Learning Type:        baseline
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               100

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## BaCP

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

all_metrics = {}
disable_loss_types = ['self', 'sup', 'all']
for disable_type in disable_loss_types:
    loss_types = {
        'self': 'self-supervised contrastive loss',
        'sup': 'supervised contrastive loss',
        'all': 'all contrastive losses',
    }

    print('='*80)
    print(f"Disabling {loss_types[disable_type]} for this training cycle.")
    print('='*80)

    bacp_training_args = BaCPTrainingArguments(
        model_name=MODEL_NAME,
        model_task=MODEL_TASK,
        batch_size=BATCH_SIZE_CNN,
        optimizer_type_and_lr=('sgd', 0.1),
        pruning_type='magnitude_pruning',
        target_sparsity=TARGET_SPARSITY_LOW,
        sparsity_scheduler='cubic',
        finetuned_weights=finetuned_weights,
        learning_type='bacp_pruning_b3',
        db=False,
        log_epochs=False,
    )
    bacp_trainer = BaCPTrainer(bacp_training_args)
    bacp_trainer.disable = disable_type
    if True:
        bacp_trainer.train()

    bacp_trainer.generate_mask_from_model()
    training_args = TrainingArguments(
        model_name=bacp_trainer.model_name,
        model_task=bacp_trainer.model_task,
        batch_size=bacp_trainer.batch_size,
        optimizer_type_and_lr=('adamw', 0.0001),
        pruner=bacp_trainer.get_pruner(),
        pruning_type=bacp_trainer.pruning_type,
        target_sparsity=bacp_trainer.target_sparsity,
        epochs=50,
        finetuned_weights=bacp_trainer.save_path,
        finetune=True,
        learning_type="bacp_b3_finetune",
        db=False,
        log_epochs=False,
    )
    trainer = Trainer(training_args)
    if True:
        trainer.train()

    metrics = trainer.evaluate()
    print_statistics(metrics, trainer)

    all_metrics[disable_type] = metrics

save_object(all_metrics, f'{MODEL_NAME}_{MODEL_TASK}_{bacp_trainer.pruning_type}_{bacp_trainer.target_sparsity}_b3_metrics.pkl')

Disabling self-supervised contrastive loss for this training cycle.


[TRAINER] Image size: 32
[ERROR] Could not load weights: ./research/resnet50/cifar10/resnet50_cifar10_baseline.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded successfully
[TRAINER] Initialized BaCP models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.1)
[TRAINER] No scheduler initialized
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/resnet50/cifar10/resnet50_cifar10_magnitude_pruning_0.95_bacp_pruning_b3.pt


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Total Loss: 7.4489 | Avg PrC Loss: 3.5383 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.5503 | Avg CE Loss: 0.3603 | Model Sparsity: 0.4636

[BaCP] weights saved!


...

Epoch [4/5]: Avg Total Loss: 13.9411 | Avg PrC Loss: 3.4925 | Avg SnC Loss: 7.5287 | Avg FiC Loss: 2.8355 | Avg CE Loss: 0.0844 | Model Sparsity: 0.9424

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: 13.9150 | Avg PrC Loss: 3.4925 | Avg SnC Loss: 7.5112 | Avg FiC Loss: 2.8308 | Avg CE Loss: 0.0804 | Model Sparsity: 0.9424



Retraining Epoch [2/10]: Avg Total Loss: 13.9065 | Avg PrC Loss: 3.4904 | Avg SnC Loss: 7.5093 | Avg FiC Loss: 2.8293 | Avg CE Loss: 0.0775 | Model Sparsity: 0.9424



Retraining Epoch [3/10]: Avg Total Loss: 13.9080 | Avg PrC Loss: 3.4920 | Avg SnC Loss: 7.5076 | Avg FiC Loss: 2.8296 | Avg CE Loss: 0.0787 | Model Sparsity: 0.9424

[BaCP] weights saved!


Retraining epoch [4/10]:  17%|█▋        | 14/83 [00:05<00:23,  2.93it/s, Loss=13.9, PrC Loss=3.48, SnC Loss=7.48, FiC Loss=2.83, CE Loss=0.0657]

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning_b1',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args, lambdas=None)
if True:
    bacp_trainer.train()
    print_dynamic_lambdas_statistics(bacp_trainer)

bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_b1_finetune",
    db=False,
)
trainer = Trainer(training_args)
if True:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

save_object(
    bacp_trainer.lambda_history, 
    f'{MODEL_NAME}_{MODEL_TASK}_{bacp_trainer.pruning_type}_{bacp_trainer.target_sparsity}_lambda_history.pkl'
    )

[TRAINER] Image size: 32
[ERROR] Could not load weights: ./research/resnet50/cifar10/resnet50_cifar10_baseline.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded successfully
[TRAINER] Initialized BaCP models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.1)
[TRAINER] No scheduler initialized
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/resnet50/cifar10/resnet50_cifar10_magnitude_pruning_0.97_bacp_pruning_b1.pt
[LOGGER] Log file created at location: ./log_records/resnet50/cifar10/bacp_pruning_b1/magnitude_pruning/0.97/run_1.log


Training Epoch [1/5]:   1%|          | 1/83 [00:01<02:15,  1.66s/it, Loss=7.62, PrC Loss=3.6, SnC Loss=0, FiC Loss=3.43, CE Loss=0.594]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Epoch [1/5]: Avg Total Loss: 7.2755 | Avg PrC Loss: 3.4377 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.4236 | Avg CE Loss: 0.4142 | Model Sparsity: 0.4734

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: 6.1858 | Avg PrC Loss: 3.1238 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 2.8219 | Avg CE Loss: 0.2401 | Model Sparsity: 0.4734



...

Epoch [5/5]: Avg Total Loss: 1.2531 | Avg PrC Loss: 0.1654 | Avg SnC Loss: 0.6876 | Avg FiC Loss: 0.1607 | Avg CE Loss: 0.2392 | Model Sparsity: 0.97

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: 1.1980 | Avg PrC Loss: 0.1585 | Avg SnC Loss: 0.6461 | Avg FiC Loss: 0.1541 | Avg CE Loss: 0.2393 | Model Sparsity: 0.97



Retraining Epoch [2/10]: Avg Total Loss: 1.1464 | Avg PrC Loss: 0.1520 | Avg SnC Loss: 0.6080 | Avg FiC Loss: 0.1479 | Avg CE Loss: 0.2384 | Model Sparsity: 0.97



Retraining Epoch [3/10]: Avg Total Loss: 1.0974 | Avg PrC Loss: 0.1462 | Avg SnC Loss: 0.5742 | Avg FiC Loss: 0.1423 | Avg CE Loss: 0.2347 | Model Sparsity: 0.97



KeyboardInterrupt: 

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning_b1',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args, lambdas=None)
if True:
    bacp_trainer.train()
    print_dynamic_lambdas_statistics(bacp_trainer)

bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_b1_finetune",
    db=False,
)
trainer = Trainer(training_args)
if True:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

save_object(
    bacp_trainer.lambda_history, 
    f'{MODEL_NAME}_{MODEL_TASK}_{bacp_trainer.pruning_type}_{bacp_trainer.target_sparsity}_lambda_history.pkl'
    )